# Examples of usage of the package `planetary2d`

## Importing the package

In [1]:
# You can import the whole package
import planetary2d

In [2]:
# Or you can import specific utilities
from planetary2d import SystemOfBodies, Animator, load_json_data, Randomizer

## General notice to usage
The function `load_json_data` serves to provide an object of type `SystemOfBodies` with
data from a JSON file processed to a custom Python data structure.

The class `SystemOfBodies` serves then to provide an easy and comfortable way of setting the
parameters of a system of bodies and of interating the object through the time-discretization steps.

The class `Animator` serves to provide a user-friendly object which can be used to parametrize the
required animation over a system of bodies passed to it and to create or save the animation as needed.

Beside the mentioned classes, there is also a class `Randomizer` for generation of random initial conditions of
systems of bodies (the data) or a random systems of bodies (the objects themselves). The process of randomization
can be parametrized.

## Using the function `load_json_data`

In [ ]:
# Data for initial conditions of the system of bodies
# can be loaded from JSON file if it is properly structured
data_valid1 = load_json_data("./examples-valid-001.json")
data_valid2 = load_json_data("./examples-valid-002.json")
# If it is not, the function raises the following exception:
try:
    data_invalid = load_json_data("./examples-invalid.json")
except RuntimeError as err:
    print(f"Caught a runtime error: {str(err)}")
    

In [ ]:
# The representation of the data loaded by `load_json_data` is
# the Python `dict` with a similar structure as that of the JSON
print(type(data_valid1))
print(data_valid2)


## Using the class `SystemOfBodies`

In [ ]:
# A system of bodies cannot be created without data as it does not make much sense
try:
    system_of_bodies = SystemOfBodies()
except TypeError as er:
    print(str(er))


In [6]:
# The valid creation of the object follows
system_of_bodies1 = SystemOfBodies(data_valid1)
# Do not omit the possibility of creating the data in the code
system_of_bodies_in_code = SystemOfBodies({'FirstBody': {'position': [1, 0],
                                                         'velocity': [7, 0.5],
                                                         'mass': 10e11},
                                           'SecondBody': {'position': [7, -5.1],
                                                          'velocity': [0.1, 35],
                                                          'mass': 16.5e13}})


See the docstring to the constructor of the class.

In [7]:
# It is important to note that the data itself says nothing about the units
# the values are measured in. Therefore, the standard SI system is used by default,
# but it can be customized to other units of time, space and mass. This is an important
# point because gravitational forces are calculated using the gravitational constant which
# must be adjusted to specific units to work correctly with the data.
system_of_bodies1 = SystemOfBodies(data_valid1, time_units='millisecs', space_units='mm')


In [8]:
# No less an important value is the `base_interval` which (in the units of time used)
# expresses the length of the period which the time discretization is broken into.
system_of_bodies1 = SystemOfBodies(data_valid1, base_interval=0.1)


In [9]:
# If no history of positions is needed to be tracked, it can be disabled:
system_of_bodies2 = SystemOfBodies(data_valid2, no_hist=True)


There are even several methods in the class.

In [10]:
# It is possible to use a several methods to manually control
# the changes of the system of bodies
system_of_bodies1.update_accelerations()  # Calculates new accelerations from the current state of the system
system_of_bodies1.update_positions()  # Calculates new positions from the current state of the system
system_of_bodies1.update_velocities()  # Calculates new velocities from the current state of the system


In [ ]:
# However, it is more convenient to use an iterator over the object,
# especially, in a for-each loop (e.g. the iterability is used in animating the system).
# Note that the iteration ends after `system_of_bodies.limit` is reached
# But the `system_of_bodies` is not affected
system_of_bodies1 = SystemOfBodies(data_valid1, limit=5)

print(system_of_bodies1.pos[0])  # `system_of_bodies` before iteration

for system_state in system_of_bodies1:
    print(system_state.pos[0])  # Positions of the first body in the system_of_bodies1 when the system iterates
                                # (in the complex notation)

print(system_of_bodies1.pos[0])  # `system_of_bodies` after iteration


## Using the class `Animator`

In [12]:
# An animator cannot be created without parameters
# There are two required positional arguments
animator = Animator(system_of_bodies1, limit=10)


The first argument is the system of bodies to animate and
the second one sets the number of iterations (repretitions of
the base interval) to go through in the animation.

In [13]:
# The class provides large variety of settings
# which are documented in the docstrings.
# One of more important arguments is the `title` which sets the title
# to be shown on top of the animation or the `frame_rate` which sets
# how fast the change of figures will be
animator0 = Animator(system_of_bodies1, 300, title="My Custom Animation", frame_rate=40)


The class `Animator` provides two methods:

In [ ]:
# One for showing the animation
# which takes no arguments
animator7 = Animator(SystemOfBodies(data_valid1, base_interval=86_400), 1000, "My Custom Animation", frame_rate=100)
animator7.show()

In [15]:
# And the other for saving the animation
# which takes at least one argument which defines the path
# to the output file
animator7.save("./examples-save-video-000.gif")

There are three output formats of video supported (`gif`, `mp4` and `avi`). Nevertheless,
only the `gif` is supported without `ffmpeg` installed in the system.

## Using a class `Randomizer`

In [16]:
# A randomizer can be created without any arguments
randomizer = Randomizer()


In [17]:
# However, if you prefer a customization of the randomizer, you can pass
# arguments specifying the intervals within which the values should range
randomizer2 = Randomizer(bodies_count_range=(100, 200),
                         positions_range=((-400.5, 301.3), (-302.17, 405.78)))


The class `Randomizer` provides two methods:

In [ ]:
# The first is based on generating data as a dictionary
# which can be passed to the constructor of the `SystemOfBodies`
dict_data = randomizer.generate_data()
print(dict_data)


In [ ]:
# The second method is only a shorthand for generating
# the dictionary and creating the system of bodies in one line.
# Note that it takes any keyword argument of the constructor of
# `SystemOfBodies` and passes it to the constructor
system_of_bodies4 = randomizer.generate_bodies_system(base_interval=0.1, no_hist=True)
print(system_of_bodies4.Δt)  # This is the base_interval !
print(system_of_bodies4.no_hist)  # This is the no_hist !


## A few examples of the versatility of the framework

Example 1

In [ ]:
# Create a system of bodies from a file
bodies_sys = SystemOfBodies(load_json_data("./examples-valid-001.json"), base_interval=86_400)
# Create the animator
planets_clrs = ['yellow', 'red', 'red', 'green',
                'orange', 'orange', 'gray', 'blue', 'blue']
animator1 = Animator(bodies_sys, 1000, frame_rate=100,
              traj_colors=planets_clrs, bdy_colors=planets_clrs, label_color='white',
              bdy_sizes=[40., 2., 2., 10., 10., 40., 30., 20., 20.],
              xlimits=(-5e12, 5e12), ylimits=(-1e12, 1e12), title="Animation example 1")
# Animate interactively
animator1.show()
# Save the animation
animator1.save("./examples-save-video-001.gif")


Example 2

In [ ]:
# Create a system of bodies from a file
bodies_sys2 = SystemOfBodies(load_json_data("./examples-valid-001.json"), base_interval=86_400)
# Create the animator
planets_clrs = ['yellow', 'red', 'red', 'green',
                'orange', 'orange', 'gray', 'blue', 'blue']
animator2 = Animator(bodies_sys2, no_units=True, limit=1000, frame_rate=100,
              traj_colors=planets_clrs, bdy_colors=planets_clrs, label_color='black',
              bdy_sizes=[40., 2., 2., 10., 10., 40., 30., 20., 20.],
              xlimits=(-5e12, 5e12), ylimits=(-1e12, 1e12), show_axes=False,
              dark_bg=False, title="Animation example 2")
# Animate interactively
animator2.show()
# Save the animation
animator2.save("./examples-save-video-002.gif")


Example 3

In [22]:
# Create a randomizer
randomizer3 = Randomizer((7, 50))
# Create a system of bodies the random data
bodies_sys3 = SystemOfBodies(randomizer3.generate_data(), base_interval=86_400)
# Create the animator
animator3 = Animator(bodies_sys3, no_units=True, limit=1000, frame_rate=100,
              traj_colors='red', bdy_colors='orange', label_color='white',
              show_axes=True, dark_bg=True, title="Animation example 3")
# Animate interactively
animator3.show()
# Save the animation
animator3.save("./examples-save-video-003.gif")


Example 4

In [ ]:
# Create a randomizer
randomizer4 = Randomizer((7, 50), velocities_range=((-10, 10), (-10, 10)))
# Create a system of bodies the random data
bodies_sys4 = randomizer4.generate_bodies_system(base_interval=86_400)
# Create the animator
animator4 = Animator(bodies_sys4, limit=1000, frame_rate=100,
              traj_colors='red', bdy_colors='orange', label_color='white',
              show_grid=False, title="Animation example 4")
# Animate interactively
animator4.show()
# Save the animation
animator4.save("./examples-save-video-004.gif")
